In [1]:
import logging, os
logging.disable(logging.WARNING)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

import tensorflow as tf

from pathlib import Path

import keras_tuner as kt

from importlib import reload
import hyper_model_creator
import keras_model_creator
import sentinel_utils

In [2]:
trials_dir = Path('trials', 'hyperband_resnet')
trial_metric = 'val_weightedf2score'

In [3]:
callbacks = [
    tf.keras.callbacks.TensorBoard(trials_dir.joinpath('board')),
    tf.keras.callbacks.EarlyStopping(
        monitor=trial_metric, patience=20, mode='max'
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor=trial_metric, factor=0.5, patience=8, min_lr=1e-6,
    ),
]

In [ ]:
reload(hyper_model_creator)

hypermodel = hyper_model_creator.BuildHyperModel()

tuner = kt.Hyperband(
    hypermodel,
    objective=kt.Objective(trial_metric, direction='max'),
    directory=trials_dir.parent,
    project_name=trials_dir.name,
    hyperband_iterations=3,
    max_epochs=100,
    overwrite=False,
)
tuner.search(
    verbose=1, 
    callbacks=callbacks
)

Trial 467 Complete [02h 02m 29s]
val_weightedf2score: 0.6919596791267395

Best val_weightedf2score So Far: 0.6974008679389954
Total elapsed time: 9d 11h 31m 32s

Search: Running Trial #468

Value             |Best Value So Far |Hyperparameter
True              |True              |class_weight
2017_2018_2019    |2017_2018_2019    |training_years
l1l2              |l1l2              |kernel_regularizer
0.3               |0.1               |spatial_dropout
leaky_relu        |relu              |activation
4                 |4                 |pool_size
0.1               |0.5               |dropout
True              |True              |bias_initializer
0.01              |0.0001            |learning_rate
binary_focal_cr...|binary_crossent...|loss_function
100               |34                |tuner/epochs
34                |12                |tuner/initial_epoch
2                 |2                 |tuner/bracket
2                 |1                 |tuner/round
0460              |0456      

/home/pj/miniconda3/envs/tf/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 102 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Epoch 35/100
3550/3550 ━━━━━━━━━━━━━━━━━━━━ 260s 69ms/step - accuracy: 0.5597 - auc: 0.9450 - loss: 0.0654 - prc: 0.8543 - precision: 0.7898 - recall: 0.7193 - weightedf1score: 0.7421 - weightedf2score: 0.7275 - val_accuracy: 0.5179 - val_auc: 0.9141 - val_loss: 0.0830 - val_prc: 0.7881 - val_precision: 0.7278 - val_recall: 0.6883 - val_weightedf1score: 0.6947 - val_weightedf2score: 0.6903 - learning_rate: 0.0100
Epoch 36/100
 733/3550 ━━━━━━━━━━━━━━━━━━━━ 2:38 56ms/step - accuracy: 0.5617 - auc: 0.9450 - loss: 0.0654 - prc: 0.8545 - precision: 0.7930 - recall: 0.7187 - weightedf1score: 0.7430 - weightedf2score: 0.7274

In [ ]:
# best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]

# best_hp.values['units'] = 128
# best_hp.values['dropout'] = 0.3

# model = tuner.hypermodel.build(best_hp)
# model.fit(x_train, y_train, epochs=10, validation_data=(x_val, y_val))

In [ ]:
# tuner.results_summary(num_trials=2)

In [ ]:
# %load_ext tensorboard
# tb_path = str(trials_dir.joinpath('board'))
# %tensorboard --logdir "$tb_path"